Things to do
- Preprocess images
- Load images
- Set up data for training
- Set up feature learning
- Add classification
- Receive video feed
- Process video feed



In [89]:
import keras
import numpy as np


### Setting the path to the dataset

In [90]:

import pathlib

data_dir = pathlib.Path("/kaggle/")
print(data_dir)

/kaggle


### Creating the dataset

##### About the dataset
We chose to split the test folder into 80% for testing and 20% for validation


In [91]:
#TODO: We should consider trying out different batch sizes
import tensorflow as tf

data = []
labels = []

batch_size = 30
img_height = 32
img_width = 32
img_size = (img_height, img_width)
epochs = 10

train_ds = tf.keras.utils.image_dataset_from_directory(
    "./../kaggle/Train/",
    validation_split=0.2,
    subset="training",
    label_mode='categorical',
    seed=505,
    image_size=img_size,
    shuffle=True,
    batch_size=batch_size
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    "./../kaggle/Train/",
    validation_split=0.2,
    subset="validation",
    label_mode='categorical',
    seed=505,
    shuffle=True,
    image_size=img_size,
    batch_size=batch_size
)

# print(validation_ds)
#
num_classes = len(train_ds.class_names)
class_names = train_ds.class_names

import matplotlib.pyplot as plt

# For visualizing images in one batch
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(32):
#         ax = plt.subplot(15, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(train_ds.class_names[labels[i]])
#         plt.axis("off")
#


Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


### Standardizing the data
#### About
In this section we will standardize the images since images comes in all types shapes.



This is for rescaling the images from RGB channel [0, 255] to [0, 1]

In [92]:
#We should verify that the images actually go from [0, 255]
normalization_layer = tf.keras.layers.Rescaling(1. / 255)


### Setting up performance

In [93]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Training the model

In [94]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal",
                                   input_shape=(img_height,
                                                img_width,
                                                3)),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

model = tf.keras.Sequential([
    ##Do we really need the normalizastion section?
    #normalization_layer,
    data_augmentation,
    tf.keras.layers.Rescaling(1.0 / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(rate=.3),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [30,43] and labels shape [1290]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits
 (defined at /usr/local/lib/python3.8/dist-packages/keras/backend.py:5113)
]] [Op:__inference_train_function_365301]

Errors may have originated from an input operation.
Input Source operations connected to node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:
In[0] sparse_categorical_crossentropy/Reshape_1 (defined at /usr/local/lib/python3.8/dist-packages/keras/backend.py:5109)	
In[1] sparse_categorical_crossentropy/Reshape (defined at /usr/local/lib/python3.8/dist-packages/keras/backend.py:3561)

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/py

### Visualize the training

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Testing/Evaluating the model

In [ ]:
#Predicting with the test data


test_ds = tf.keras.utils.image_dataset_from_directory(
    "./../kaggle/Test/",
    image_size=img_size,
    batch_size=batch_size,
)

res = model.evaluate(test_ds)
print("test loss, test acc:", res)

#### Predicting the trained model

In [ ]:
img = tf.keras.preprocessing.image.load_img('../kaggle/Test/0/11812.png', target_size=img_size)
input_arr = tf.keras.preprocessing.image.img_to_array(img)
#input_arr = np.expand_dims(input_arr, axis=0)
input_arr = np.array([input_arr])  # Convert single image to a batch.

predict = model.predict(input_arr)
score = tf.nn.softmax(predict[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
)